# Extract complete Hourly water level data from downloaded zip folder

In [1]:
####  Import necessary libraries
import os
import zipfile
import pandas as pd
import datetime
import re

fpath='C:\\Data\\daily rainfall and hourly water level complete data'
zipfilename='Station-2A300-WL1544195714.zip'

stnname1 = re.findall('^Station-(.*)-WL',zipfilename)
wlzipnum1 = re.findall('-WL(.*).zip',zipfilename)
stnname = stnname1[0]
wlzipnum = wlzipnum1[0]

now=datetime.datetime.now()
extractdata=fpath+'\\eve_data'               ## file path to get extracted files
dwl_fpath='C:\Data'                               ## file path for data collected from extracted zipfiles
rcsvhWL=wlzipnum+'_water_level.csv'               ## ReadCSVHourlyWaterLevel[rcsvhWL]

#### Extract data from zipped folder
os.chdir(fpath)     # change directory where WL data is downloaded
zfwl = zipfile.ZipFile(zipfilename)     ## create file name as per the standard format (E.g. Station-W2-1544019002.zip)
zfwl.namelist()
zfwl.extractall(fpath)     ## file path where extracted files need to be stored
os.chdir(extractdata)     ## Now, read extracted csv file

df_hWL = pd.read_csv(rcsvhWL, skiprows = 5, usecols = [0,1], header = None, 
                     names = ['Time', 'Daily WL(feet NAVD)'], 
                     sep = ',', index_col = False)                    ## to load hourly water level csv file into the dataframe

df_hWL['Daily WL(meters NAVD)']=(df_hWL['Daily WL(feet NAVD)']*0.3048).round(2)

tframe_from= df_hWL.iloc[0]['Time'][:10]
tframe_to=df_hWL.iloc[-1]['Time'][:10]
tstamp=str(now)[11:13]+str(now)[14:16]+str(now)[17:19] + str(now)[-7:]     ## WriteCSVDailyWaterLevel[wcsvdWL]
wcsvhWL="hWL["+stnname+"]["+tframe_from+" to "+tframe_to+"]"+tstamp+".csv"

os.chdir(dwl_fpath)     ## to save values in new csv file

df_hWL.to_csv(wcsvhWL,index = False, columns = ['Time','Daily WL(feet NAVD)','Daily WL(meters NAVD)'])

# Extract specified period of hourly water level data to prepare daily water level data

In [ ]:
####  Import necessary libraries
import os
import pandas as pd
import datetime
import re

#### file path for extracted csv file of hwl
getfilepath='C:\\Data'
os.chdir(getfilepath)
getWLcsv='hWL[EDEN_6][2006-07-10 to 2019-02-07]233954.253826.csv'

#### Specified hour in a day [spf_hrs]
getTime='08:00:00'
firstDate='2016-01-01'
lastDate='2018-01-08'

#### regular expression to extract station name
station=re.findall('^hWL\[(.*?)]',getWLcsv)
station=station[0]
dataset=re.findall('\]\[(.*?)]',getWLcsv)

print('Overall length of data for Station '+station+' is:\n'+dataset[0])
df_hWL=pd.read_csv(getWLcsv,sep=',', names=['Time','DailyWL(ft NAVD)','DailyWL(m NAVD)'])
print('Data starts from '+df_hWL.iloc[0]['Time'][:10]+
      ' to '+df_hWL.iloc[-1]['Time'][:10])

#### to pull values as per userdefined timeframe (i.e. DailyWL at specified time)
df_dWL=df_hWL[(df_hWL['Time'].str.contains(getTime))]

#### to select rows in a DataFrame between two values
df_dWL=df_dWL[(df_dWL['Time']>firstDate)&(df_dWL['Time']<=lastDate)]
df_dWL=df_dWL.reset_index(drop = True)

#### to save values in new csv file
#### Write CSV Daily Water Level [wcsvdWL]
now=datetime.datetime.now()
timeStamp=str(now)[11:13]+str(now)[14:16]+str(now)[17:19]+str(now)[-7:]
setdWLfile="dWL["+station+"]["+firstDate+" to "+lastDate+"]"+timeStamp+".csv"
setfilepath='C:\\Data'
os.chdir(setfilepath)
df_dWL.to_csv(setdWLfile,index=False,columns=['Time','DailyWL(ft NAVD)','DailyWL(m NAVD)'])